<a href="https://colab.research.google.com/github/jjbmsda/kaggle/blob/main/Feb_Tabular_Playground_Competition/Feb_Tabular_Playground_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Feb Tabular Playground Competition

/content/drive/MyDrive/Colab Notebooks/Feb Tabular Playground Competition


In [4]:
pip install category_encoders

     |████████████████████████████████| 81kB 3.8MB/s 


In [5]:
import numpy as np 
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import category_encoders as ce
        

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**데이터 로드**

In [14]:
df_train = pd.read_csv('train.csv')
display(df_train.head())

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [15]:
df_test = pd.read_csv('test.csv')
display(df_test.head())

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,A,B,A,C,B,D,A,E,E,G,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,A,B,A,C,B,D,A,E,C,L,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,A,B,A,C,B,D,A,E,C,F,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,A,A,B,A,B,D,A,E,E,F,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,A,B,A,A,B,B,A,E,E,I,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [16]:
df_sample = pd.read_csv('sample_submission.csv')
display(df_sample.head())

,id,target
0,0,0.5
1,5,0.5
2,15,0.5
3,16,0.5
4,17,0.5


**데이터 전처리**

In [17]:
df_train.drop("id", axis=1, inplace=True) #train 데이터에서 id 열을 버립니다. inplace가 True면 복사본을 반환하지 않습니다.
df_test.drop("id", axis=1, inplace=True) #test 데이터에서 id 열을 버립니다.

In [19]:
cat_labels = [f"cat{i}" for i in range(10)]

In [20]:
onehot_encoder = ce.one_hot.OneHotEncoder() #OneHotEncoder를 정의합니다.
onehot_encoder.fit(pd.concat([df_train[cat_labels], df_test[cat_labels]], axis=0)) #train 데이터와 test 데이터를 합친 후 OneHotEncoding을 실행합니다.
train_ohe = onehot_encoder.transform(df_train[cat_labels]) #train 데이터에 OneHotEncoding을 적용한 train_ohe를 만듭니다.
test_ohe = onehot_encoder.transform(df_test[cat_labels]) #test 데이터에 OneHotEncoding을 적용한 test_ohe를 만듭니다.
train_ohe.columns = [f"OHE_{col}" for col in train_ohe] #OneHotEncoder는 각 레이블의 이름을 정해주지 않기 때문에, 레이블의 이름을 정해줍니다. 
test_ohe.columns = [f"OHE_{col}" for col in test_ohe] #레이블의 이름을 정해줍니다. 

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [21]:
numerical_labels = [f"cont{i}" for i in range(14)] #cont0부터 cont13까지 14개의 숫자로 이루어진 레이블을 정의합니다.

In [22]:
train_x = pd.concat([df_train[numerical_labels],train_ohe], axis=1) #OneHotEncoding을 한 문자 label과 숫자 label을 합쳐 최종 train 데이터를 만듭니다.

In [23]:
test_x = pd.concat([df_test[numerical_labels],test_ohe], axis=1) #최종 train 데이터를 만듭니다.

In [24]:
train_y = df_train["target"] #train 데이터의 Target 값을 train_y로 정의해줍니다. 

**모델 훈련**

In [25]:
folds = KFold(n_splits=5, shuffle=True, random_state=2021) #5 fold의 KFold를 정의합니다. random_state는 임의대로 정의하여도 됩니다. 

In [26]:
lgb_params = dict({'objective': 'regression', #LGBM 모델의 다양한 parameter들을 정의합니다. 이 문제는 회귀 문제이므로 regression으로 정의합니다. 
 'metric': 'rmse', #이 대회에서는 평가에 RMSE를 사용합니다. 
 'verbosity': -1, #모델 훈련 상태를 얼마만큼 알려줄 것인지 설정합니다. 
 'learning_rate': 0.01, #learning rate를 설정합니다
 'early_stopping_round' : 200, #200번의 훈련 동안 모델의 성능에 향상이 없으면 학습을 중지합니다. 
 'num_iterations' : 20000}) #최대 20000번 훈련합니다. 

In [27]:
oof_preds = np.zeros_like(train_y) #validation의 예측값을 저장할 DataFrame을 만듭니다.
models = [] #5개의 모델을 저장할 list를 만듭니다.
for tr_idx, va_idx in folds.split(train_x): #5번 상호 검증을 수행하며 훈련할 for 반복문을 만듭니다. 
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx] #예측에 사용할 x 값들에서 validation 데이터를 분리합니다.
    tr_y, va_y = train_y[tr_idx], train_y[va_idx] #예측할 데이터인 y 값들에서 validation 데이터를 분리합니다.
            
    lgb_train_dataset = lgb.Dataset(tr_x, tr_y) #train 데이터를 LGBM용 데이터셋으로 변환합니다. 자세한 내용은 공식 문서를 참조하세요. https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Dataset.html
    lgb_valid_dataset = lgb.Dataset(va_x, va_y) #test 데이터를 LGBM용 데이터셋으로 변환합니다.
    model = lgb.train(lgb_params, lgb_train_dataset, valid_sets=[lgb_valid_dataset], verbose_eval=100) #LGBM 모델을 훈련합니다.
    models.append(model) #각 모델을 위에서 정의한 모델 리스트에 입력합니다.
    oof_pred = model.predict(va_x) #validation 데이터를 예측합니다.
    oof_preds[va_idx] = oof_pred #예측한 값을 위에서 만든 DataFrame에 입력합니다.

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[100]	valid_0's rmse: 0.865812
[200]	valid_0's rmse: 0.858065
[300]	valid_0's rmse: 0.854028
[400]	valid_0's rmse: 0.851472
[500]	valid_0's rmse: 0.849695
[600]	valid_0's rmse: 0.848408
[700]	valid_0's rmse: 0.847412
[800]	valid_0's rmse: 0.846638
[900]	valid_0's rmse: 0.846033
[1000]	valid_0's rmse: 0.845574
[1100]	valid_0's rmse: 0.845271
[1200]	valid_0's rmse: 0.845041
[1300]	valid_0's rmse: 0.844883
[1400]	valid_0's rmse: 0.844761
[1500]	valid_0's rmse: 0.844662
[1600]	valid_0's rmse: 0.844575
[1700]	valid_0's rmse: 0.844503
[1800]	valid_0's rmse: 0.844463
[1900]	valid_0's rmse: 0.844425
[2000]	valid_0's rmse: 0.844367
[2100]	valid_0's rmse: 0.844335
[2200]	valid_0's rmse: 0.844285
[2300]	valid_0's rmse: 0.844244
[2400]	valid_0's rmse: 0.844215
[2500]	valid_0's rmse: 0.844202
[2600]	valid_0's rmse: 0.844189
[2700]	valid_0's rmse: 0.844163
[2800]	valid_0's rmse: 0.844142
[2900]	valid_0's rmse: 0.844121
[3000]	valid_0's r

In [28]:
np.sqrt(mean_squared_error(df_train.target, oof_preds)) #각 모델로 산출한 validation 데이터의 RMSE 점수를 출력하여 모델의 성능을 확인합니다.

0.8446788870093547

In [29]:
preds = [] #예측값을 저장할 DataFrame을 만듭니다.
for model in models: #저장했던 모델들을 하나씩 예측하는 for 반복문을 만듭니다.
    pred = model.predict(test_x) #각 모델을 하나씩 예측합니다.
    preds.append(pred) #각 예측값을 위에서 만든 DataFrame에 입력합니다.
preds = np.mean(preds, axis=0) #각 예측값의 평균을 내어 최종 예측값을 완성합니다.

In [30]:
sub = df_sample.copy() #submission 샘플을 복사합니다.
sub["target"] = preds #샘플의 target column에 최종 예측값을 입력합니다.

sub.to_csv("submission_baseline(simple parameter).csv", index=False) #제출을 위한 csv 파일을 생성합니다.

sub.head() #최종 제출 파일의 앞부분을 확인합니다.

,id,target
0,0,7.606025
1,5,7.748879
2,15,7.598841
3,16,7.449088
4,17,7.296246
